In [1]:
import pickle
import os
from openpyxl import Workbook
import requests
import urllib
import hashlib
import json
import time
# from pypinyin import pinyin

In [2]:
baidu_ak = 'xkQNCsGZgBBYa2vAvEOyHn7qEvFlsdPu'
baidu_sk = 'DHcj9OshlO4r2Z1kLL7k6W8gu1f6Pqun'
province_names = [
                # 直辖市
                '直辖市', 
                # 自治区类别
                  '新疆自治区', '西藏自治区', '宁夏自治区', '内蒙古自治区', 
                  '广西自治区',
                # 常规省份
                  '黑龙江', '吉林', '辽宁', '河北', '山东', '江苏', '安徽',
                  '浙江', '福建', '广东', '海南', '云南', '贵州', '四川',
                  '湖南', '湖北', '河南', '山西', '陕西', '甘肃', '青海',
                  '江西', '台湾', 
                # 特别行政区
                  '特别行政区']
city_names=[
    # 直辖市
    ['北京','天津','上海','重庆'],
    # 自治区类别
    ['乌鲁木齐', '克拉玛依'],
    ['拉萨'],
    ['银川', '石嘴山', '吴忠', '固原', '中卫'],
    ['呼和浩特', '包头', '乌海', '赤峰', '通辽', '鄂尔多斯', '呼伦贝尔', 
        '巴彦淖尔', '乌兰察布'],
    ['南宁', '柳州', '桂林', '梧州', '北海', '崇左', '来宾', '贺州', '玉林', 
        '百色', '河池', '钦州', '防城港', '贵港'],
    # 常规省份
    ['哈尔滨', '大庆', '齐齐哈尔', '佳木斯', '鸡西', '鹤岗', '双鸭山', '牡丹江', 
        '伊春', '七台河', '黑河', '绥化'],
    ['长春', '吉林', '四平', '辽源', '通化', '白山', '松原', '白城'],
    ['沈阳', '大连', '鞍山', '抚顺', '本溪', '丹东', '锦州', '营口', '阜新', 
        '辽阳', '盘锦', '铁岭', '朝阳', '葫芦岛'],
    ['石家庄', '唐山', '邯郸', '秦皇岛', '保定', '张家口', '承德', '廊坊', '沧州',
         '衡水', '邢台'],
    ['济南', '青岛', '淄博', '枣庄', '东营', '烟台', '潍坊', '济宁', '泰安', 
        '威海', '日照', '莱芜', '临沂', '德州', '聊城', '菏泽', '滨州'],
    ['南京', '镇江', '常州', '无锡', '苏州', '徐州', '连云港', '淮安', '盐城', 
        '扬州', '泰州', '南通', '宿迁'],
    ['合肥', '芜湖', '蚌埠', '淮南', '马鞍山', '淮北', '铜陵', '安庆', '黄山', 
        '阜阳', '宿州', '滁州', '六安', '宣城', '池州', '亳州', '潜山'],
    ['杭州', '嘉兴', '湖州', '宁波', '金华', '温州', '丽水', '绍兴', '衢州', 
        '舟山', '台州'],
    ['福州', '厦门', '泉州', '三明', '南平', '漳州', '莆田', '宁德', '龙岩'],
    ['广州', '深圳', '汕头', '惠州', '珠海', '揭阳', '佛山', '河源', '阳江', 
        '茂名', '湛江', '梅州', '肇庆', '韶关', '潮州', '东莞', '中山', '清远', 
        '江门', '汕尾', '云浮'],
    ['海口', '三亚'],
    ['昆明', '曲靖', '玉溪', '保山', '昭通', '丽江', '普洱', '临沧'],
    ['贵阳', '六盘水', '遵义', '安顺'],
    ['成都', '绵阳', '德阳', '广元', '自贡', '攀枝花', '乐山', '南充', '内江', 
        '遂宁', '广安', '泸州', '达州', '眉山', '宜宾', '雅安', '资阳'],
    ['长沙', '株洲', '湘潭', '衡阳', '岳阳', '郴州', '永州', '邵阳', '怀化', 
        '常德', '益阳', '张家界', '娄底'],
    ['武汉', '襄樊', '宜昌', '黄石', '鄂州', '随州', '荆州', '荆门', '十堰', 
        '孝感', '黄冈', '咸宁'],
    ['郑州', '洛阳', '开封', '漯河', '安阳', '新乡', '周口', '三门峡', '焦作', 
        '平顶山', '信阳', '南阳', '鹤壁', '濮阳', '许昌', '商丘', '驻马店'],
    ['太原', '大同', '忻州', '阳泉', '长治', '晋城', '朔州', '晋中', '运城', 
        '临汾', '吕梁'],
    ['西安', '咸阳', '铜川', '延安', '宝鸡', '渭南', '汉中', '安康', '商洛', 
        '榆林'],
    ['兰州', '天水', '平凉', '酒泉', '嘉峪关', '金昌', '白银', '武威', '张掖', 
        '庆阳', '定西', '陇南'],
    ['西宁'],
    ['南昌', '九江', '赣州', '吉安', '鹰潭', '上饶', '萍乡', '景德镇', '新余', 
        '宜春', '抚州'],
    ['台北', '台中', '基隆', '高雄', '台南', '新竹', '嘉义'],
    # 特别行政区
    ['香港', '澳门']
]

In [3]:
city_data_dict = dict()
for prov_ind, prov_name in enumerate(province_names):
    for city_ind, city_name in enumerate(city_names[prov_ind]):
        with open(os.path.join('bilibili-comment-by-city', city_name+'.pkl'), 'rb') as f:
            city_data = pickle.load(f)
        city_data_dict[city_name] = city_data

In [17]:
def extract_statistic(city_data: dict):
    interaction_sum = 0
    comment_num = 0
    for video_url, value in city_data.items():
        interaction_sum += value[2]['interactionStatistic']['userInteractionCount']
        comment_num += len(value[3])
    return interaction_sum, comment_num


def get_address_info_baidu(address, baidu_ak, baidu_sk):
    # 以get请求为例http://api.map.baidu.com/geocoding/v3/?address=百度大厦&output=json&ak=yourak
    queryStr = '/geocoding/v3/?address={address}&output=json&ak={ak}'.format(address=address, ak=baidu_ak)

    # 对queryStr进行转码，safe内的保留字符不转换
    encodedStr = urllib.parse.quote(queryStr, safe="/:=&?#+!$,;'@()*[]")

    # 在最后直接追加上yoursk
    rawStr = encodedStr + baidu_sk

    # md5计算出的sn值7de5a22212ffaa9e326444c75a58f9a0
    # 最终合法请求url是http://api.map.baidu.com/geocoder/v2/?address=百度大厦&output=json&ak=yourak&sn=7de5a22212ffaa9e326444c75a58f9a0
    sn = hashlib.md5(urllib.parse.quote_plus(rawStr).encode('utf-8')).hexdigest()
    res = requests.get(url='http://api.map.baidu.com/geocoding/v3/', 
                   params={'address':address,
                          'output': 'json',
                          'ak': baidu_ak,
                          'sn': sn})
    if res.status_code == 200:
        res_json = json.loads(res.text)
        return res_json
    return None


def get_address_location_baidu(address, baidu_ak, baidu_sk):
    # 以get请求为例http://api.map.baidu.com/geocoding/v3/?address=百度大厦&output=json&ak=yourak
    queryStr = '/geocoding/v3/?address={address}&output=json&ak={ak}'.format(address=address, ak=baidu_ak)

    # 对queryStr进行转码，safe内的保留字符不转换
    encodedStr = urllib.parse.quote(queryStr, safe="/:=&?#+!$,;'@()*[]")

    # 在最后直接追加上yoursk
    rawStr = encodedStr + baidu_sk

    # md5计算出的sn值7de5a22212ffaa9e326444c75a58f9a0
    # 最终合法请求url是http://api.map.baidu.com/geocoder/v2/?address=百度大厦&output=json&ak=yourak&sn=7de5a22212ffaa9e326444c75a58f9a0
    sn = hashlib.md5(urllib.parse.quote_plus(rawStr).encode('utf-8')).hexdigest()
    res = requests.get(url='http://api.map.baidu.com/geocoding/v3/', 
                   params={'address':address,
                          'output': 'json',
                          'ak': baidu_ak,
                          'sn': sn})
    if res.status_code == 200:
        res_json = json.loads(res.text)
        if res_json['status'] == 0:
            return res_json['result']['location']
    return None

In [6]:
extract_statistic(city_data_dict[city_names[0][3]])

(13996578, 19478)

In [11]:
wb = Workbook()
ws = wb.create_sheet('bilibili-city-info-full', 0)
ws.cell(row=1, column=1, value='City Name')
ws.cell(row=1, column=2, value='Longitude')
ws.cell(row=1, column=3, value='Latitude')
ws.cell(row=1, column=4, value='User Interaction Count')
ws.cell(row=1, column=5, value='Comment Count')
base_index = 2
no_location_cities = {}
for prov_ind, prov_name in enumerate(province_names):
    for city_ind, city_name in enumerate(city_names[prov_ind]):
        ws.cell(row=city_ind+base_index, column=1, value=city_name)

        # Get location info from baidu api
        location = get_address_location_baidu(city_name, baidu_ak, baidu_sk)
        time.sleep(1)
        if location is not None:
            ws.cell(row=city_ind+base_index, column=2, value=location['lng'])
            ws.cell(row=city_ind+base_index, column=3, value=location['lat'])
        else:
            no_location_cities[(prov_ind, city_ind)]=city_name
            print(city_name+' location retrieve error !')
        stat = extract_statistic(city_data_dict[city_name])
        ws.cell(row=city_ind+base_index, column=4, value=stat[0])
        ws.cell(row=city_ind+base_index, column=5, value=stat[1])
    base_index += len(city_names[prov_ind])
wb.save('bilibili-city-info-full.xlsx')

襄樊 location retrieve error !
香港 location retrieve error !


In [ ]:
[_[1] for _ in city_data_dict[city_names[3]].values()]

In [21]:
get_address_info_baidu('HongKong', baidu_ak, baidu_sk)

{'status': 1, 'msg': 'Internal Service Error:无相关结果', 'results': []}